In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 初始化权值的函数
def weight_variable(shape):
    # 生成一个截断的正态分布
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1)) 

# 初始化偏值
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape)) 

# # 根据样本样式定义创建占位符（1列）
x = tf.placeholder(tf.float32,[None,784]) # 每张图的像素点信息： 28*28 = 784
y = tf.placeholder(tf.float32,[None,10])  # 十个数字

# 改变 x 的格式转为4D的向量 [batch,`img_height`,`img_width`, `img_channels`].(复原为原来图片的样子)
# MNIST 图片二维数据为28*28 ，一维数据格式为 784 长度的数组
# -1 表示类型有 x 动态决定
x_image = tf.reshape(x,[-1,28,28,1])

# ---------------------------------------------------------------
# 定义卷积结构
# ---------------------------------------------------------------

# 初始化第一个卷基层
convl = tf.layers.conv2d(
  inputs = x_image,       # input: 4-D tensor[`batch`, `img_height`,`img_width`, `img_channels`]
  filters = 32,           # 生成32个（kernel_size类型）的特征平面
  kernel_size = [5, 5],   # 卷积核大小  2*2  ， Tip: 当长、宽相等时可以写成：kernel_size = 5
  padding = "same",       # SAME (保证输出的shape与输入的一致) 、 VALID(该例中，VALID模式将导致输出 tensor 的 shape 为 24*24) 
  activation = tf.nn.relu # 激活函数
)
# 进行池化
pool1 = tf.layers.max_pooling2d(
    inputs = convl,     # 将卷基层作为输入
    pool_size = [2, 2], # 池化窗口大小。 Tip: 当长、宽相等时可以写成： pool_size = 2
    strides = [2, 2]    # 步长。 Tip: 当长、宽相等时可以写成： strides = 2       
)

# 初始化第二个卷基层
conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size= 5,
      padding="same",
      activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)

# ---------------------------------------------------------------
# 分析 
# ---------------------------------------------------------------
# 1. 28*28 的图片第一次卷积后还是 28*28 （因为采用 same 模式进行采样）
# 2. 第一次池化后，则变为 14*14 （因为池化层步长为 2*2）
# 3. 则接下来，第二次对 14*14 进行卷积后还是 14*14 
# 4. 第二次池化后，则变为 7*7 

# ---------------------------------------------------------------
# 定义神经网络结构
# ---------------------------------------------------------------

# 把池化层2的输出扁平化为1维
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
# 初始化第一个神经网络层， 神经元数：1024, 激活函数： relu
dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
# prediction = tf.layers.dense(inputs=dense1, units=10, activation=tf.nn.softmax)
# keep_prob 用来表示神经元的输出概率

# 神经网络最后一层（模型层）
keep_prob = tf.placeholder(tf.float32) 
dropout = tf.layers.dropout(
      inputs=dense1, rate=keep_prob, training = True)
logits = tf.layers.dense(inputs=dropout, units=10)

# 预测结果
prediction = tf.nn.softmax(logits)

# 交叉熵
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction))

# 使用AdamOptimizer进行优化
train =  tf.train.AdamOptimizer(1e-4).minimize(loss)

# 结果存放在一个布尔型的列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))# argmax 返回一维张量中最大值的所以在位置
# 求准确率
# cast 把布尔型列表转换为float32， 如[true.true.false] =》 [1,1,0] ，那么准确率的值即为66.6%
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess: 
    # 初始化全部变量
    sess.run(tf.global_variables_initializer())
    # 训练21个周期
    for epoch in range(21):
        # n_batch ： 一共有多少个批次
        for batch in range(n_batch):
            # 保存batch_size张图片的数据与标签
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        # 用测试集的图片及标签求得准确率
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter "+ str(epoch) + ", Texting acc " + str(acc))




ImportError: No module named 'tensorflow.contrib.learn'